## Reordering Columns in Pandas

Since you can't do select(column_name,everything()) in pandas....

In this example, we reorder the column in a pandas dataframe so column c is first

In [13]:
import pandas as pd
# create example dataset
example = pd.DataFrame({'a': [1,2,3],'b': [10,11,12],'c': [45,55,65]})

In [14]:
example

a   b   c
0  1  10  45
1  2  11  55
2  3  12  65

In [15]:
# Grab column names, remove a column name, and insert that column name at the beginning
col_names = example.columns.tolist()
col_names.remove('c')

# Note - if you want to move column c to another position, you can specify a different index
# you can anchor the column on another column by using col_names.index() to extract 
# the index of another column

col_names.insert(0,'c')
# Use the reordered column names to reindex this dataset
example = example.reindex(columns=col_names)

In [16]:
example

c  a   b
0  45  1  10
1  55  2  11
2  65  3  12

## Frequency Counts

The purpose of this function is to count the values or combination of values in an entire dataset or a subselection of the columns in a dataset. I have created this function because I couldn't find a pandas function that can count both missing and non-missing values while also looking at a combination of values (multiple columns)

In [68]:
import random
import string
import numpy as np
from IPython.display import display

# column names (optional) defined as string or list of strings
# Returns frequency table that includes missing values for df
def tidy_count(df,columns=None,count_col='n',return_df=False,n=None):
    # subset df by columns if specified
    if columns != None:
        if type(columns) != list:
            columns = [columns]
        df = df[columns]
        
    col_vals = df.columns.values # all column values
        
    # Find all distinct values in the dataset for both data values
    # and column names, store in the list allvals
    pds_allvals = pd.Series(col_vals) 
    for col in col_vals:
        pds_allvals = pd.concat([pds_allvals,df[str(col)]]).drop_duplicates()
    allvals = pds_allvals.tolist()
    #print(allvals)
    
    filler_val=None
    # generate a random string for filler_val until it is 
    # not something that occurs in our data
    loop_iters=1
    while filler_val == None or filler_val in allvals:
        filler_val = ''.join(random.choices(string.digits + string.ascii_letters, k=np.random.randint(4,12)))  
        loop_iters += 1
        if loop_iters > 8:
            print('Error: Cannot find unique string to fill NA values.')
            break
    #print(filler_val)
    
    # reset count column name until it is unique among column names
    loop_iters=1
    while count_col in col_vals:
        if count_col == 'n':
            count_col = count_col + 'n'
        else:
            count_col = count_col + '_'
        
        if loop_iters > 8:
            print('Error: Cannot find unique string for counter column.')
            break    
        loop_iters += 1

    freq_table = df.fillna(filler_val).groupby([str(x) for x in col_vals]).size().\
            reset_index(name=count_col).\
         replace(filler_val,np.NaN).sort_values(count_col,ascending=False)
    
    freq_table['percent'] = freq_table[count_col] / sum(freq_table[count_col])
    
    # limit frequency table to first n rows if specified
    if n != None:
        freq_table = freq_table.head(n)
    
    # Formatted pandas display for jupyter notebooks
    df_display = display(freq_table.style.hide_index().\
                format({'n': '{:,}','percent': '{0:.1%}'}))
    
    if return_df == False:
        df_display
    else:
        df_display
        return(freq_table)

## Test

In [69]:
car_names = pd.DataFrame({'name' : ['Batmobile','Toyota Corolla','Bike','Bike','Batmobile'],
            'hp': [1000,120,np.nan,np.nan,900]})
car_attr = pd.DataFrame({"name": ["Bike","Toyota Corolla"], "color": ["blue","red"]})
cars = car_names.merge(car_attr,how='left',on='name')

In [11]:
a = tidy_count(cars,return_df=True)

In [5]:
a

name      hp color  n  percent
2            Bike     NaN  blue  2      0.4
0       Batmobile   900.0   NaN  1      0.2
1       Batmobile  1000.0   NaN  1      0.2
3  Toyota Corolla   120.0   red  1      0.2

In [10]:
tidy_count(cars,['hp','color'])

In [14]:
tidy_count(cars)

## Bigger Frequency Counts

In [48]:
import seaborn as sns
import numpy as np
planets = sns.load_dataset("planets")

In [79]:
import numpy as np
planets_method = planets.sample(frac=.40,random_state=42)
planets_year = planets.sample(frac=.20,random_state=84)
planets_method['method'] = np.nan
planets_year['year'] = np.nan

planets_combi = pd.concat([planets.\
                    drop(planets_method.index.union(planets_year.index)),
                          planets_method,planets_year]).sort_index()

In [80]:
planets_combi.sample(10)

method  number  orbital_period    mass  distance    year
573  Radial Velocity       2       613.80000  1.8500     68.35  2010.0
318              NaN       2        18.35700  0.0390     52.03  2013.0
834          Transit       2         2.58664     NaN       NaN     NaN
821          Transit       7        91.93913     NaN    780.00  2013.0
560  Radial Velocity       2      2891.00000  1.5020     15.89  2003.0
562  Radial Velocity       1      1038.10000  1.9000     54.23  2007.0
29               NaN       1             NaN     NaN     45.52  2005.0
451              NaN       3       559.40000  3.0000     52.83  2007.0
309              NaN       6         4.31230  0.0126     12.83  2008.0
82               NaN       2       125.94000  0.1700     20.10  2013.0

In [84]:
tidy_count(planets_combi,['method','year'],n=10)